In [0]:
df_bronze = spark.table("resume_bronze")
df_bronze.show(truncate=False)


+-----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|resume_file            |resume_text                                                                                                                                                                                                                                                                                                                                                                                           |
+-----------------------+---------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
skills_list = [
    "python",
    "sql",
    "machine learning",
    "deep learning",
    "nlp",
    "pyspark",
    "databricks",
    "spark",
    "pandas",
    "numpy",
    "spacy",
    "etl",
    "data analysis",
    "cosine similarity"
]


In [0]:
from pyspark.sql.functions import udf, lower
from pyspark.sql.types import ArrayType, StringType

def extract_skills(text):
    text = text.lower()
    found_skills = []
    for skill in skills_list:
        if skill in text:
            found_skills.append(skill)
    return found_skills

extract_skills_udf = udf(extract_skills, ArrayType(StringType()))


In [0]:
df_silver = df_bronze \
    .withColumn("resume_text_clean", lower(df_bronze.resume_text)) \
    .withColumn("extracted_skills", extract_skills_udf("resume_text_clean"))

df_silver.select("resume_file", "extracted_skills").show(truncate=False)


+-----------------------+-------------------------------------------------------------------------------+
|resume_file            |extracted_skills                                                               |
+-----------------------+-------------------------------------------------------------------------------+
|Resume_Akash_Verma.pdf |[python, sql, pyspark, databricks, spark, etl]                                 |
|Resume_Ananya_Gupta.pdf|[python, sql, machine learning, nlp, spacy]                                    |
|Resume_Karthik_R.pdf   |[python, sql]                                                                  |
|Resume_Rohan_Sharma.pdf|[python, sql, machine learning, nlp, pyspark, databricks, spark, data analysis]|
|Resume_Sneha_Iyer.pdf  |[python, sql, machine learning, pandas, data analysis]                         |
+-----------------------+-------------------------------------------------------------------------------+



In [0]:
df_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("resume_silver")
